# Preprocessamento:

+ Criar variável que identifique o anotador

+ Mover tags que começam com token ' ' (vazio)

+ Remover linhas com '\n' seguidos

+ REGEX:
    + Garantir letra e números onde tamanho for maior que 1.
    + Passar múltiplos símbolos para outra linha. Exemplo:  §3º -->  § \n 3 \n º
    + Remover pontuação de centenas dos números. Exemplo: 12.200 --> 12200

+ Visualização das sentenças com displacy (from spacy import displacy) 

+ Incluir POS tagging.

+ Tranformar o dado $x_i$ em um $x'_i$ que incorpora os 2 últimos e próximos tokens.

# Classificador

+ Visualização: Separar o conjunto de test em 2 ou 3 arquivos e visualizar o que o modelo classificou e o que os anotadores classificaram (separar por id do anotador).

+ Parâmetros utilizados no classificador.

+ Analisar o formato dos dados que tem maior acerto e menor acerto tambem.

+ Para criar um contexto no erro imprimir 10 palavras antes e depois de dois erros.


# Instruções para os anotadores:

+ Atentar à marcação de tags que envolve espaço 

+ Atentar para não incluir espaço no início da Tag

# Organização do diretório: 

+ Manter toda a análise em somente um diretório

+ Formato do diretório com os datasets: /resources/dataset/

+ Notebooks:
    + '01 - Processamento.ipynb'
        + Gerar 'treino.csv' e teste.csv' para processamento
    + '02 - [CRF].ipynb' - Criando o modelo
        + Usar arquivo dos dados preprocessados gerado pelo notebook 1.
        + Gerar modelo (xxx.model)
    + '03 - Metricas.ipynb'

In [1]:
import os
import re 
import glob
import time
import numpy as np
import pandas as pd
from collections import Counter

import scipy.stats
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import make_scorer
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.feature_extraction.text import HashingVectorizer

In [2]:
# Encontra todos os csv's dentro das pastas de 'mock'

extension = 'csv'
all_filenames = [i for i in glob.glob('NER/*/**/***/****/*****.{}'.format(extension))]

# NER/NER_EXPORT/161704902/[PRATICA_ETAPA_1]/Documentos

In [3]:
all_filenames[:5]

['NER/NER_EXPORT/181300055/[PRATICA_ETAPA_1]/Documentos/20141030_RE_541090_271775268.ner.csv',
 'NER/NER_EXPORT/181300055/[PRATICA_ETAPA_1]/Documentos/20180411_HC_138057_314087281.ner.csv',
 'NER/NER_EXPORT/181300055/[PRATICA_ETAPA_1]/Documentos/20170921_HC_147683_312770301.ner.csv',
 'NER/NER_EXPORT/181300055/[PRATICA_ETAPA_1]/Documentos/20180503_Pet_7074_314257052.ner.csv',
 'NER/NER_EXPORT/181300055/[PRATICA_ETAPA_1]/Documentos/20180509_HC_135415_314294988.ner.csv']

# Preprocessamento 

In [4]:
# Cria uma tag de inicio e fim de arquivo em cada 'csv' antes de apendar todos eles.

frames = []
for all_files in all_filenames:
    df = pd.read_csv(all_files,delimiter=';', na_values='NaN') # Lê o arquivo
    df['Tag'].iloc[0] , df['Tag'].iloc[-1] = ['INICIO_ARQ', 'FIM_ARQ'] # Altera a primeira e ultima Tag desse csv
    frames.append(df) # Adiciona esse dataframe no 'dataframe maior'
    
combined_csv = pd.concat(frames).reset_index(drop=True)
combined_csv.to_csv("combined_csv.csv",index=False,encoding='utf-8') # Cria um arquivo com todas as anotações.
combined_csv['Token'] = combined_csv['Token'].astype('str')

In [5]:
combined_csv.head(15), combined_csv.tail(20)

(             Token         Tag
 0           EMENTA  INICIO_ARQ
 1                :           O
 2                            O
 3   CONSTITUCIONAL           O
 4                .           O
 5                            O
 6       TRIBUTÁRIO           O
 7                .           O
 8                            O
 9          IMPOSTO           O
 10                           O
 11              DE           O
 12                           O
 13           RENDA           O
 14               .           O,
                                  Token       Tag
 15446379                                       O
 15446380                      dezembro         O
 15446381                                       O
 15446382                            de         O
 15446383                                       O
 15446384                          2015         O
 15446385                             .         O
 15446386                                       O
 15446387                            

In [6]:
print("Número de linhas dos arquivos concatenados:", len(combined_csv['Tag']))

Número de linhas dos arquivos concatenados: 15446399


In [7]:
combined_csv[-10:] # Conferindo se o index foi resetado

,Token,Tag
15446389,Ministro,B_Pessoa
15446390,,I_Pessoa
15446391,TEORI,I_Pessoa
15446392,,I_Pessoa
15446393,ZAVASCKI,I_Pessoa
15446394,\n,I_Pessoa
15446395,id,O
15446396,:,O
15446397,,O
15446398,20160201_HC_132184_308418258,FIM_ARQ


# Encontra parágrafo duplo no arquivo. Uma opção de separar por sentenças.

In [8]:
a_df = combined_csv #Simplifica o nome do arquivo para a função nao ficar grande demais.
starts = a_df[a_df['Token']=='\n'].index & a_df[a_df['Token'].shift(-1)=='\n'].index #Identifica os paragrafos duplos
print(u'Padrões(sentenças) encontrados:', len(starts))

Padrões(sentenças) encontrados: 159578


In [ ]:
%%time

combined_csv['Sentence #'] = 'Sentence'

combined_csv['Sentence #'][:starts[0]+2] = 'Sentence %d'%(1) # Primeira sentença
combined_csv['Sentence #'][starts[-1]+2:] = 'Sentence %d'%(len(starts)+1) # Última sentença

for i in range(1,len(starts)):
    combined_csv['Sentence #'][starts[i-1]+2:starts[i]+2] = 'Sentence %d'%(i+1) 

combined_csv.head(), combined_csv.tail()

In [ ]:
# Número de sentenças
len(combined_csv['Sentence #'].unique())

# Atualiza a Tag que termina com 'Doutrinador' para 'Doutrina'

In [ ]:
combined_csv.Tag.unique()

In [ ]:
# Strip o final 'dor' de todo o DataFrame (formato extremo)
# combined_csv.Tag = combined_csv.Tag.str.rstrip('dor')

In [ ]:
# indices de onde a tag ocorre
indx = combined_csv[combined_csv.Tag.str.endswith('Doutrinador')].index.values

### Rodar apenas uma vez

In [ ]:
%%time
# Demorado e custoso
for i in range(len(indx)):
    combined_csv.Tag.iloc[indx[i]] = combined_csv.Tag.iloc[indx[i]].rstrip('dor')

In [ ]:
combined_csv.Tag.unique()

# Remove enter duplo depois de criar as sentenças

In [ ]:
# Teste para ver os casoso onde ocorre enter duplo.
for i in range(len(starts)):
    print(combined_csv.iloc[starts[i]:starts[i]+2][:5])

In [ ]:
# Cria um array com as posições a serem retiradas.
pos = []
for i in range(len(starts)):
    pos.append(starts[i])
    pos.append(starts[i]+1)
pos[:5]

In [ ]:
# Remove as linhas do dataframe e reseta os índices.
combined_csv = combined_csv.drop(pos).reset_index(drop=True)

In [ ]:
# Confirma se a remoção foi bem sucedida.
combined_csv.head(15)

# Mover B_  com Token vazio para linha abaixo

In [ ]:
# 'begins' identifica as situações onde a Tag começa com 'B_' e o Token é vazio, uma situação onde 
# o anotador começou a marcação de um espaço vazio gerando a inconsistância.

begins = combined_csv[(combined_csv['Token']==' ') & 
                      (combined_csv['Tag'].str.startswith('B_'))].index.values

In [ ]:
begins[0], len (begins)

In [ ]:
%%time
# Rodar apenas uma vez
for i in range(len(begins)):
    combined_csv.Tag.iloc[begins[i]+1] = combined_csv.Tag.iloc[begins[i]] #Acertar a Tag do Token para começar
                                                                          #sem espaço
    combined_csv.Tag.iloc[begins[i]] = 'O' #Marca o espaço vazio como 'O'

In [ ]:
i, n = 1, 10
combined_csv.iloc[begins[i]-n+7:begins[i]+n]

# Tratar marcações que incluem vírgula no final da marcação

In [ ]:
df_teste = combined_csv.copy()

In [ ]:
# df_teste.iloc[2].shift(+1)

df_teste['Token'].shift(-1)

In [ ]:
df_teste.iloc[2]

In [ ]:
# Índice das posições onde ocorre o fim da marcação em uma vírgula
inx = df_teste[(df_teste.Tag.str.startswith('I')) & (df_teste.Token == ',') & (df_teste.Tag.shift(-1) == 'O')].index.values

In [ ]:
# Conferindo a ocorrência
i = 9
df_teste.iloc[inx[i]-5:inx[i]+10]

In [ ]:
df_teste = df_teste.drop(inx).reset_index(drop=True)

In [ ]:
# Conferindo se funcionou
i = 9
df_teste.iloc[inx[i]-20:inx[i]+15]

In [ ]:
combined_csv = df_teste.copy()

In [ ]:
combined_csv[(combined_csv.Tag.str.startswith('B')) & (combined_csv.Token == ' ')]

In [ ]:
list(combined_csv.Tag.unique())

# IMPORTANTE: Esse último passo é necessário para salvar todas as alterações feitas no preprocessamento.

In [ ]:
# Salvando o processamento feito nos dados
combined_csv.to_csv("preprocessados.csv",index=False,encoding='utf-8')

# <center> $\color{red}{\text{TESTES FALHOS}}$ </center>

## Tratar caracteres especiais

In [ ]:
# Criando um dataframe teste para não alterar o arquivo principal
df_teste = combined_csv.copy()

In [ ]:
# Identifica os tokens que começam com '§' e são maiores que 1 pois queremos pegar os casos onde
# '§' está associado ao numero do paragrafo.
espec_carac = list(df_teste[(df_teste.Token.str.startswith('§')) & (df_teste.Token.str.len()>1)].Token.unique())
espec_carac

In [ ]:
df_teste[df_teste.Token.str.match(espec_carac[0])]

### Teste com duas sentenças

In [ ]:
df_senten = df_teste[df_teste['Sentence #'] == 'Sentence 310'].append(df_teste[df_teste['Sentence #'] == 'Sentence 1589'])

df_senten.reset_index(drop=True,inplace=True)

In [ ]:
for i in range(len(df_senten)):
    if any(carac in df_senten.Token.iloc[i] for carac in espec_carac) == True:
        line = df_senten.iloc[i]
        splt = list(line.Token)
        
        if line.Tag.startswith('I_'):
            if len(line.Token) == 2:
                #Para tamanho 2 temos que transformar uma linha em duas
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()

                df_senten1 = pd.concat([df_senten.iloc[:i], line0, line1, df_senten.iloc[i+1:]]).reset_index(drop=True)
            
            if len(line.Token) == 3:
                #Para tamanho 3 temos que transformar uma linha em três
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line2 = line.copy()
                line2.Token = splt[2]

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()
                line2 = pd.DataFrame(line2).transpose()

                df_senten1 = pd.concat([df_senten.iloc[:i], line0, line1, line2, df_senten.iloc[i+1:]]).reset_index(drop=True)

        if line.Tag.startswith('B_'):
            if len(line.Token) == 2:
                #Para tamanho 2 temos que transformar uma linha em duas
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line1.Tag = line.Tag.replace('B_','I_')

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()

                df_senten1 = pd.concat([df_senten.iloc[:i], line0, line1, df_senten.iloc[i+1:]]).reset_index(drop=True)
            if len(line.Token) == 3:
                #Para tamanho 3 temos que transformar uma linha em três
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line1.Tag = line.Tag.replace('B_','I_')
                line2 = line.copy()
                line2.Token = splt[2]
                line2.Tag = line.Tag.replace('B_','I_')

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()
                line2 = pd.DataFrame(line2).transpose()

                df_senten1 = pd.concat([df_senten.iloc[:i], line0, line1, line2, df_senten.iloc[i+1:]]).reset_index(drop=True)

In [ ]:
# Vendo se o novo dataframe (df_senten1) está alterado em relação ao dataframe anterior (df_senten).

In [ ]:
(df_senten1.tail(10))

In [ ]:
df_senten.tail(10)

### Para o conjunto de teste (com todas as sentenças)

In [ ]:
for i in range(len(df_teste)):
    if any(carac in df_teste.Token.iloc[i] for carac in espec_carac) == True:
        line = df_teste.iloc[i]
        splt = list(line.Token)
        
        if line.Tag.startswith('I_'):
            if len(line.Token) == 2:
                #Para tamanho 2 temos que transformar uma linha em duas
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()

                df_senten1 = pd.concat([df_teste.iloc[:i], line0, line1, df_teste.iloc[i+1:]]).reset_index(drop=True)
            
            if len(line.Token) == 3:
                #Para tamanho 3 temos que transformar uma linha em três
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line2 = line.copy()
                line2.Token = splt[2]

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()
                line2 = pd.DataFrame(line2).transpose()

                df_senten1 = pd.concat([df_teste.iloc[:i], line0, line1, line2, df_teste.iloc[i+1:]]).reset_index(drop=True)

        if line.Tag.startswith('B_'):
            if len(line.Token) == 2:
                #Para tamanho 2 temos que transformar uma linha em duas
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line1.Tag = line.Tag.replace('B_','I_')

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()

                df_senten1 = pd.concat([df_teste.iloc[:i], line0, line1, df_teste.iloc[i+1:]]).reset_index(drop=True)
            if len(line.Token) == 3:
                #Para tamanho 3 temos que transformar uma linha em três
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line1.Tag = line.Tag.replace('B_','I_')
                line2 = line.copy()
                line2.Token = splt[2]
                line2.Tag = line.Tag.replace('B_','I_')

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()
                line2 = pd.DataFrame(line2).transpose()

                df_senten1 = pd.concat([df_teste.iloc[:i], line0, line1, line2, df_teste.iloc[i+1:]]).reset_index(drop=True)

In [ ]:
df_senten1[(df_senten1.Token.str.startswith('§')) & (df_senten1.Token.str.len()>1)].head()

# Aparentemente, não está funcionando.

## Remove pontuação dos números

In [ ]:
# Criando um DataFrame onde todas os tokens possuem pontuação 
df_pont = combined_csv[(combined_csv.Token.str.contains("""[.]""")) & (combined_csv.Token.str.len()>1)]

In [ ]:
df_pont.shape

In [ ]:
# Tokens de tamanho maior que 1.
df_pont.head(30)

In [ ]:
# combined_csv.iloc[2722-3:2722+5]
df_pont[(df_pont.Tag.str.endswith('Precedente'))]

In [ ]:
df_pont.Tag.value_counts()

### Não foi possível chegar em um consenso sobre qual pontuação deveria ser removida e como localizá-la no df.

## Caso onde 'ADV' aparece ao final do Token

In [ ]:
# No meio do preprocessamento foi identificado tokens que terminam com 'adv' e estão colados ao nome do
# advogado referente ao caso.
# Tokens que terminan com 'adv'.

In [ ]:
adv = combined_csv[combined_csv.Token.str.endswith('ADV')]#.reset_index(drop=True)
adv.head()#, adv.shape

In [ ]:
# Desconfiança que o nome do advogado está agregado à palavra 'adv'

adv[adv.Token.str.len() >3]
adv_pos = adv[adv.Token.str.len() >3].index.values

k = 2 # Vê as linhas antes e depois do k-ésimo ocorrido .
combined_csv.iloc[adv_pos[k]-3:adv_pos[k]+3] # Olhando para as linhas anteriores e posteriores o acontecimento.
# combined_csv.iloc[adv_pos[0]].Token[-3:] #Separando a parte 'adv'

In [ ]:
print("Quantidade de vezes que esse caso ocorre em todos os arquivos:",len(adv_pos))